In [1]:
#6->1->6

## Imports

In [2]:
import random

import pennylane as qml
from pennylane import numpy as np

from pennylane.optimize import AdamOptimizer

import torch
from torchvision import datasets, transforms

C:\Users\tomut\anaconda3\envs\qhack2022\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\tomut\anaconda3\envs\qhack2022\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [3]:
import sys
sys.path.append("..") # Adds higher directory to python modules path

from qencode.initialize import setAB_amplitude, setAux, setEnt
from qencode.encoders import e4_entangled_zoom
from qencode.training_circuits import swap_t
from qencode.qubits_arrangement import QubitsArrangement

from qencode.utils.mnist import get_dataset

## Get Data

In [4]:
input_data = get_dataset(img_width=8, img_height=8, train=True)
print("Original data set size:", len(input_data))

# Select only the pictures with numbers 0 or 1. (jus to compare with literature)
filtered_data = [image for image in input_data if image[1] in [0, 1]]
input_data = filtered_data
print("Final data set szize:", len(input_data))


Original data set size: 60000
Final data set szize: 12665


## Training node

In [5]:
shots = 2500
nr_trash=5
nr_latent=1
nr_ent=2
ent_state= [1 / np.sqrt(2), 0, 0, 1 / np.sqrt(2)]
reinit_state= [0 for i in range(2**(nr_trash+nr_ent))]
reinit_state[0]= 1 / np.sqrt(2)
reinit_state[3]= 1 / np.sqrt(2)

spec = QubitsArrangement(nr_trash, nr_latent, nr_swap=1, nr_ent=nr_ent, nr_aux=nr_trash+nr_ent)
print("Qubits:", spec.qubits)

#set up the device 
dev = qml.device("default.qubit", wires=spec.num_qubits)

Qubits: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [6]:
@qml.qnode(dev)
def training_circuit_e4(init_params,encoder_params, reinit_state):
    # Initialization
    
    setAB_amplitude(spec, init_params)
    setAux(spec, reinit_state)
    setEnt(spec, inputs=ent_state)

    #encoder 
    e4_entangled_zoom(encoder_params, spec)
        

    #swap test 
    original_qubits=[*spec.trash_qubits,*spec.ent_qubits]
    for i in spec.swap_qubits:
        qml.Hadamard(wires=i)
    for i in range(len(original_qubits)):
        qml.CSWAP(wires=[*spec.swap_qubits, spec.aux_qubits[i], original_qubits[i]])
    for i in spec.swap_qubits:
        qml.Hadamard(wires=i)

    return [qml.probs(i) for i in spec.swap_qubits]

## Training parameters 

In [7]:
epochs = 2000
learning_rate = 0.0003
batch_size = 5
num_samples = 50
nr_layers= 2

beta1 = 0.9
beta2 = 0.999
opt = AdamOptimizer(learning_rate, beta1=beta1, beta2=beta2)

In [8]:
def fid_func(output):
    # Implemented as the Fidelity Loss
    # output[0] because we take the probability that the state after the 
    # SWAP test is ket(0), like the reference state
    fidelity_loss = 1 / output[0]
    return fidelity_loss

In [9]:
def cost(encoder_params, X):
    reinit_state = [0 for _ in range(2 ** len(spec.aux_qubits))]
    reinit_state[0] = 1.0
    loss = 0.0
    for x in X:
        output = training_circuit_e4(init_params=x[0][0],
                                     encoder_params=encoder_params,
                                     reinit_state=reinit_state, )[0]
        f = fid_func(output)
        loss = loss + f
    return loss / len(X)

In [10]:
def fidelity(encoder_params, X):
    reinit_state = [0 for _ in range(2 ** len(spec.aux_qubits))]
    reinit_state[0] = 1.0
    loss = 0.0
    for x in X:
        output = training_circuit_e4(init_params=x[0][0],
                                          encoder_params=encoder_params,
                                          reinit_state=reinit_state)[0]
        f = output[0]
        loss = loss + f
    return loss / len(X)

In [11]:
def iterate_batches(X, batch_size):
    X1 = [torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits)))) for x in X]
    X2 = []
    for i in range(len(X1)):
        X2.append([X1[1], X[i][1]])
    X = X2
    random.shuffle(X)

    batch_list = []
    batch = []
    for x in X:
        if len(batch) < batch_size:
            batch.append(x)

        else:
            batch_list.append(batch)
            batch = []
    if len(batch) != 0:
        batch_list.append(batch)
    return batch_list

In [12]:
training_data = [input_data[i] for i in range(num_samples)]
test_data = [input_data[i] for i in range(num_samples,num_samples+num_samples)]

In [13]:
X_training = [[torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits))))] for x in training_data]
X_tes=[[torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits))))] for x in test_data]

In [14]:

# initialize random encoder parameters
nr_encod_qubits = len(spec.trash_qubits) + len(spec.latent_qubits)+nr_ent
nr_par_encoder =nr_par_encoder =  15 * int(nr_encod_qubits*(nr_encod_qubits-1)/2)
l1_params = np.random.uniform(size=(1, nr_par_encoder), requires_grad=True)


                              

"""
                              
nr_encod_qubits = len(spec.trash_qubits) + len(spec.latent_qubits)
nr_par_encoder =nr_par_encoder =  15 * int(nr_encod_qubits*(nr_encod_qubits-1)/2)
l3_params = np.random.uniform(size=(1, nr_par_encoder), requires_grad=True)
"""


encoder_params = [*l1_params]#*l3_params]

###  training

In [ ]:
loss_hist=[]
fid_hist=[]

loss_hist_test=[]
fid_hist_test=[]

for epoch in range(epochs):
    batches = iterate_batches(X=training_data, batch_size=batch_size)
    for xbatch in batches:
        encoder_params = opt.step(cost, encoder_params, X=xbatch)


    
    if epoch%5 == 0:
        
        loss_training = cost(encoder_params, X_training )
        fidel = fidelity(encoder_params, X_training )
        
        loss_hist.append(loss_training)
        fid_hist.append(fidel)
        print("Epoch:{} | Loss:{} | Fidelity:{}".format(epoch, loss_training, fidel))

        loss_test = cost(encoder_params, X_tes )
        fidel = fidelity(encoder_params, X_tes )
        loss_hist_test.append(loss_test)
        fid_hist_test.append(fidel)
        print("Test-Epoch:{} | Loss:{} | Fidelity:{}".format(epoch, loss_test, fidel))


    

C:\Users\tomut\anaconda3\envs\qhack2022\lib\site-packages\pennylane\_grad.py:95: UserWarning: Starting with PennyLane v0.21.0, when using Autograd, inputs have to explicitly specify requires_grad=True (or the argnum argument must be passed) in order for trainable parameters to be identified.
  warnings.warn(
C:\Users\tomut\anaconda3\envs\qhack2022\lib\site-packages\pennylane\math\multi_dispatch.py:63: UserWarning: Contains tensors of types {'torch', 'autograd'}; dispatch will prioritize TensorFlow and PyTorch over autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


Epoch:0 | Loss:1.9585827084284435 | Fidelity:0.5107077003586942
Test-Epoch:0 | Loss:1.9667798513990562 | Fidelity:0.5086409420085658
Epoch:5 | Loss:1.9278789482962209 | Fidelity:0.518942086741219
Test-Epoch:5 | Loss:1.9351860348431487 | Fidelity:0.5170629286577814
Epoch:10 | Loss:1.8982117793258653 | Fidelity:0.5271951153770629
Test-Epoch:10 | Loss:1.904867404338317 | Fidelity:0.5254388599955924
Epoch:15 | Loss:1.8691959565007161 | Fidelity:0.5356002104779766
Test-Epoch:15 | Loss:1.8748417339892387 | Fidelity:0.534065141324863
Epoch:20 | Loss:1.843005193774791 | Fidelity:0.5435093099919549
Test-Epoch:20 | Loss:1.8471447905361404 | Fidelity:0.542333554240021
Epoch:25 | Loss:1.8191785961895703 | Fidelity:0.5509550150341676
Test-Epoch:25 | Loss:1.8221159312752837 | Fidelity:0.5500582640608151
Epoch:30 | Loss:1.797782377013073 | Fidelity:0.5577978003415708
Test-Epoch:30 | Loss:1.8001796274857491 | Fidelity:0.5570006813426686
Epoch:35 | Loss:1.7793528197203665 | Fidelity:0.5638039538618953


## Rezults

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
plt.plot([x for x in range(0,len(loss_hist)*5,5)],np.array(fid_hist),label="train fid")
plt.plot([x for x in range(0,len(loss_hist)*5,5)],np.array(fid_hist_test),label="test fid")


plt.legend()
plt.title("6-2-6:fidelity e4",)
plt.xlabel("epoch")
plt.ylabel("fid")

In [ ]:
fig = plt.figure()
plt.plot([x for x in range(0,len(loss_hist)*5,5)],np.array(loss_hist),label="train loss")
plt.plot([x for x in range(0,len(loss_hist)*5,5)],np.array(loss_hist_test),label="test loss")


plt.legend()
plt.title("6-2-6:loss e4",)
plt.xlabel("epoch")
plt.ylabel("loss")

In [ ]:
name = "training_e4"

Circuit_prop={   "shots":shots, "nr_trash":nr_trash, "nr_latent":nr_latent ,"nr_ent":nr_ent ,"ent_state":ent_state }
Training_param = { "num_samples" : num_samples,
                    "batch_size" :batch_size,
                    "nr_layers":nr_layers,
                    "epochs" :epochs,
                    "learning_rate" : learning_rate ,
                    "beta1" : beta1,
                    "beta2 ":beta2,
                     "optimizer":"Adam"}

performance={"loss_hist":loss_hist, "fid_hist":fid_hist,
             "loss_hist_test":loss_hist_test, "fid_hist_test":fid_hist_test,
             "encoder_params":encoder_params}

experiment_data={"Circuit_prop":Circuit_prop,
                "Training_param":Training_param,
                "performance:":performance,
                "Name":name}

# open file for writing
f = open(name+".txt","w")
f.write( str(experiment_data) )
